In [1]:
import graphlab as gl
import graphlab.aggregate as agg
import numpy as np
import scipy.sparse as sps

In [2]:
from DataReader import DataReaderChallangePolimi2017
dr = DataReaderChallangePolimi2017(evaluation=True, rebuild=False)

Date Reader Challange Polimi 2017
Loading data from ./DumpData/DataReaderTrainTestDump
Dataset train-test loaded


In [3]:
knn = 150
n_rec = 5

train = dr.train
test = dr.test

In [4]:
print train[0]
print train[1]

[ 8890 22426 54743 ..., 32763 32763 32763]
[71058 19741 71298 ..., 87949 62773 70493]


In [9]:
training_data = gl.SFrame({"playlist_id":train[0],"track_id":train[1]})
test_data = gl.SFrame({"playlist_id":test[0],"track_id":test[1]})
target_playlists=gl.SFrame({"playlist_id":dr.getTargetPlaylists()})
target_tracks = gl.SFrame({"track_id":dr.getTargetTracks()})

In [34]:
id_t = np.concatenate([dr.albums[0],dr.artists[0]])
info_t = np.concatenate([dr.albums[1],dr.artists[1]])
side_tracks = gl.SFrame({"track_id":dr.albums[0],"album":dr.albums[1]})
#print side_tracks.head

In [35]:
ow, n_o = dr.getOwners()
side_playlists = gl.SFrame({"playlist_id":ow[0],"owners":ow[1]})
#print side_playlists.head

In [36]:
'''
model = gl.item_similarity_recommender.create(training_data,user_id='playlist_id', item_id='track_id', 
                                              similarity_type='cosine', 
                                              training_method='dense',
                                              only_top_k=knn,
                                              verbose = True)
'''

model = gl.ranking_factorization_recommender.create(training_data,
                                             user_id='playlist_id',
                                             item_id='track_id',     
                                             user_data = side_playlists,
                                             item_data = side_tracks,
                                             num_factors = 25, #default 32
                                             regularization = 1e-6, #1e-09
                                             linear_regularization = 1e-6, #1e-09
                                             num_sampled_negative_examples =3, #4
                                             binary_target = False, #False
                                             max_iterations = 50, #50
                                             #sgd_step_size = 0, #0 sample auto
                                             #random_seed = 0, #0
                                             solver = "auto" #ials,adagrag,sgd,auto
                                            )

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 964012 observations with 57561 users and 99990 items.

Data prepared in: 0.466772s

Training ranking_factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached (hard limit).

Computing final objective value and training Predictive Error.

Final objective value: 1.39262

Final training Predictive Error: 0.692672

In [37]:
recommendation = model.recommend(k=n_rec,exclude_known = True,users=target_playlists , items=target_tracks, verbose= False)
dic = {}
for rec in recommendation:
    p = int(rec["playlist_id"])
    t = int(rec["track_id"])
    if p in dic:
        dic[p].append(t)
    else:
        dic[p] = []
        dic[p].append(t)
rec = []
for p in dic.keys():
    rec.append([int(p),dic[p]])

m=dr.evaluateMAPfromRecommendations(rec)

User 0 of 10000
User 2500 of 10000
User 5000 of 10000
User 7500 of 10000
Recommender performance is: MAP = 0.0012


In [38]:
print m

0.00121630555556
